In [1]:
import os

# Check if we are in the nested 'SVM' folder
if os.getcwd().endswith('SVM'):
    os.chdir('../..')  # Go up two levels to the project root
    print("Directory changed to Project Root.")

print("Current Working Directory:", os.getcwd())

Directory changed to Project Root.
Current Working Directory: d:\college\Senior Year\Machine Learning\Material-Stream-Identification-System


In [3]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import LabelEncoder

augmentedDir = "data\\augmented"  
classes = [d for d in os.listdir(augmentedDir) if os.path.isdir(os.path.join(augmentedDir,d))]
print("Classes found:", classes)

IMG_SIZE = (128, 128)
LBP_RADIUS = 1
LBP_N_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'
HOG_PIXELS_PER_CELL = (8, 8)
HOG_CELLS_PER_BLOCK = (2, 2)
HOG_ORIENTATIONS = 9
COLOR_BINS = 16  

#  Feature Extraction
X = []
y = []


for cls in classes:
    folder = os.path.join(augmentedDir, cls)
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    print(f"Processing class '{cls}' with {len(files)} images...")
    
    for file in files:
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        # Resize
        img = cv2.resize(img, IMG_SIZE)
        
        # Convert to grayscale for HOG and LBP
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # HOG feature
        hog_feat = hog(
            gray,
            orientations=HOG_ORIENTATIONS,
            pixels_per_cell=HOG_PIXELS_PER_CELL,
            cells_per_block=HOG_CELLS_PER_BLOCK,
            block_norm='L2-Hys',
            transform_sqrt=True,
            feature_vector=True
        )
        
        # LBP feature
        lbp = local_binary_pattern(gray, LBP_N_POINTS, LBP_RADIUS, LBP_METHOD)
        (lbp_hist, _) = np.histogram(lbp.ravel(),
                                     bins=np.arange(0, LBP_N_POINTS + 3),
                                     range=(0, LBP_N_POINTS + 2))
        lbp_hist = lbp_hist.astype("float")
        lbp_hist /= (lbp_hist.sum() + 1e-6)
        
        # Color histogram (HSV)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h_hist = cv2.calcHist([hsv], [0], None, [COLOR_BINS], [0, 180])
        s_hist = cv2.calcHist([hsv], [1], None, [COLOR_BINS], [0, 256])
        v_hist = cv2.calcHist([hsv], [2], None, [COLOR_BINS], [0, 256])
        color_hist = np.concatenate([h_hist, s_hist, v_hist]).ravel()
        color_hist = color_hist / (color_hist.sum() + 1e-6)

        # merge all features
        feature_vector = np.concatenate([hog_feat, lbp_hist, color_hist])
        
        X.append(feature_vector)
        y.append(cls)

X = np.array(X)
y = np.array(y)


le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Feature extraction done!")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("y_encoded shape:", y_encoded.shape)
print("Class mapping:", dict(zip(le.classes_, le.transform(le.classes_))))


Classes found: ['cardboard', 'glass', 'metal', 'paper']
Processing class 'cardboard' with 1540 images...
Processing class 'glass' with 1540 images...
Processing class 'metal' with 1540 images...
Processing class 'paper' with 1540 images...
Feature extraction done!
X shape: (6160, 8158)
y shape: (6160,)
y_encoded shape: (6160,)
Class mapping: {np.str_('cardboard'): np.int64(0), np.str_('glass'): np.int64(1), np.str_('metal'): np.int64(2), np.str_('paper'): np.int64(3)}


In [4]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np


print("Original X shape:", X.shape)
print("Original y shape:", y.shape)


le = LabelEncoder()
y_encoded = le.fit_transform(y)
class_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Class mapping:", class_mapping)
print("y_encoded shape:", y_encoded.shape)

# split data 
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("Before scaling:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

# Standardization (Mean=0, Std=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("After scaling:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


Original X shape: (6160, 8158)
Original y shape: (6160,)
Class mapping: {np.str_('cardboard'): np.int64(0), np.str_('glass'): np.int64(1), np.str_('metal'): np.int64(2), np.str_('paper'): np.int64(3)}
y_encoded shape: (6160,)
Before scaling:
X_train shape: (4928, 8158)
X_test shape: (1232, 8158)
After scaling:
X_train shape: (4928, 8158)
X_test shape: (1232, 8158)


In [ ]:
# scaling done in previous step start training the model

In [ ]:
from sklearn.svm import SVC
import numpy as np


model = SVC( probability= True)

model.fit (X_train, y_train)

y_pred_prob = model.predict_proba(X_test)

Confidence_Threshold = 0.7
max_val = np.max(y_pred_prob)
max_val_idx = np.argmax(y_pred_prob)



y_pred_classes = [ np.argmax(prob_list) if (np.max(prob_list)>= Confidence_Threshold) else 6  for prob_list in y_pred_prob ]



ValueError: Expected 2D array, got 1D array instead:
array=[2. 1. 2. ... 3. 3. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [9]:
from sklearn.metrics import accuracy_score

print(model.score(X_test,y_test))

0.8717532467532467
